# Feature Engineering
Para consultar nuestra API, requerimos una nueva columna que indique los sentimientos de las reseñas de usuarios. Usaremos una reconocida librería de procesamiento de lenguaje natural para facilitar este proceso.

## Importaciones

In [44]:
import pandas as pd
import json
from textblob import TextBlob

## Preparación de conjuntos de datos

In [45]:
# Extraer datos desde los archivos 'parquet' con información que se preparo en la etapa de ETL
df_games = pd.read_parquet('../Datasets/steam_games.parquet')
df_items = pd.read_parquet('../Datasets/user_items.parquet')
df_reviews = pd.read_parquet('../Datasets/user_reviews.parquet')

In [46]:
df_games.columns

Index(['genres', 'item_id', 'release_year'], dtype='object')

In [47]:
df_items.columns

Index(['item_id', 'playtime_forever', 'user_id'], dtype='object')

In [48]:
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'helpful', 'recommend',
       'review'],
      dtype='object')

## Análisis de sentimientos
Se pide crear una nueva columna llamada 'sentiment_analysis' que reemplace a 'review' donde se realice un análisis de sentimiento de los comentarios con la siguiente escala:

0 si es malo,<br>
1 si es neutral o esta sin review<br>
2 si es positivo.

In [49]:
from textblob import TextBlob

# Función para clasificar el sentimiento según la escala dada
def classify_sentiment(polarity):
    if polarity < -0.2:
        return 0
    elif polarity > 0.2:
        return 2
    else:
        return 1

# Aplicar el análisis de sentimiento y crear la nueva columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: classify_sentiment(TextBlob(x).sentiment.polarity))

Finalmente, se elimina la columna de 'review'.

In [50]:
df_reviews = df_reviews.drop(columns=['review'])
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'helpful', 'recommend',
       'sentiment_analysis'],
      dtype='object')

In [51]:
#Verificamos si hay nulos
df_reviews.isnull().sum().sort_values(ascending=False)

user_id               0
user_url              0
posted                0
item_id               0
helpful               0
recommend             0
sentiment_analysis    0
dtype: int64

## Género y el año con más horas jugadas

Se crea el dataframe 'max_playtime_by_genre_year' con la información del año con más horas jugadas por género.

In [52]:
# Crear un dataframe combinando la información necesaria
combined_df = pd.merge(df_items, df_games[['item_id', 'release_year', 'genres']], on='item_id', how='left')

# Agrupar por género, año y sumar las horas jugadas
grouped = combined_df.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()

# Encontrar el año con más horas jugadas por género
max_playtime_by_genre_year = grouped.loc[grouped.groupby('genres')['playtime_forever'].idxmax()]

# Convertir la columna 'release_year' de flotante a entero
max_playtime_by_genre_year['release_year'] = max_playtime_by_genre_year['release_year'].astype(int)

# Mostrar el resultado
max_playtime_by_genre_year



,genres,release_year,playtime_forever
26,Action,2012,1085635110
59,Adventure,2011,221707756
71,Animation & Modeling,2015,1345545
75,Audio Production,2014,455463
104,Casual,2015,81708365
109,Design & Illustration,2012,1930339
116,Early Access,2013,118902893
124,Education,2013,340567
141,Free to Play,2013,146213734
158,Indie,2006,446115272


Dataframe con el usuario que acumula más horas jugadas por género

In [55]:
# Combinar los dataframes df_games y df_items en función de 'item_id'
merged_df = pd.merge(df_games, df_items, on='item_id')

# Convertir las horas jugadas a horas en lugar de minutos
#merged_df['playtime_hours'] = merged_df['playtime_forever'] / 60

# Calcular la suma total de horas jugadas por usuario y género
user_genre_playtime = merged_df.groupby(['user_id', 'genres'])['playtime_forever'].sum().reset_index()

# Encontrar el usuario que acumula más horas jugadas por género
max_playtime_per_genre = user_genre_playtime.loc[user_genre_playtime.groupby('genres')['playtime_forever'].idxmax()]


# Imprimir o trabajar con los resultados
print("Dataframe con el usuario que acumula más horas jugadas por género:")
print(max_playtime_per_genre)

Dataframe con el usuario que acumula más horas jugadas por género:
                  user_id                 genres  playtime_forever
485519            Sp3ctre                 Action           1692650
472192       REBAS_AS_F-T              Adventure           2182066
479391         ScottyG555   Animation & Modeling            168314
447836        Lickidactyl       Audio Production            109916
472195       REBAS_AS_F-T                 Casual           1224933
479393         ScottyG555  Design & Illustration            168314
18051   76561197978756659           Early Access            316969
160609  76561198059330972              Education             65427
570659         idonothack           Free to Play            808241
472200       REBAS_AS_F-T                  Indie           2401378
417826        Evilutional  Massively Multiplayer            688260
483274           Sky_Wolf          Photo Editing             21885
634706        shinomegami                    RPG           102

In [56]:
max_playtime_per_genre.columns

Index(['user_id', 'genres', 'playtime_forever'], dtype='object')

Dataframe con la acumulación de horas jugadas por año de los usuarios del dataframe 'max_playtime_per_genre'

In [57]:
# Obtener los usuarios contenidos en el dataframe obtenido en el punto 1
users_of_interest = max_playtime_per_genre['user_id'].unique()

# Filtrar el dataframe original de items para incluir solo a estos usuarios
filtered_items_df = df_items[df_items['user_id'].isin(users_of_interest)]

# Combinar los dataframes df_games y el filtrado de items en función de 'item_id'
filtered_merged_df = pd.merge(df_games, filtered_items_df, on='item_id')

# Calcular la suma total de horas jugadas por año
playtime_per_year = filtered_merged_df.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()
playtime_per_year.rename(columns={'playtime_forever': 'playtime_total_minutes'}, inplace=True)

# Convertir las horas jugadas a horas en lugar de minutos
playtime_per_year['playtime_total_hours'] = playtime_per_year['playtime_total_minutes'] / 60

# Resumen de horas jugadas por año de los usuarios seleccionados por género
summary_playtime_per_year = playtime_per_year.groupby(['user_id', 'release_year'])['playtime_total_hours'].sum().reset_index()

print("\nDataframe con la acumulación de horas jugadas por año de los usuarios contenidos en el punto 1:")
print(summary_playtime_per_year)


Dataframe con la acumulación de horas jugadas por año de los usuarios contenidos en el punto 1:
               user_id  release_year  playtime_total_hours
0    76561197978756659        1998.0              0.000000
1    76561197978756659        1999.0              0.133333
2    76561197978756659        2000.0              0.000000
3    76561197978756659        2001.0              0.000000
4    76561197978756659        2003.0              0.000000
..                 ...           ...                   ...
292        shinomegami        2013.0          18116.283333
293        shinomegami        2014.0          18525.700000
294        shinomegami        2015.0          18367.966667
295        shinomegami        2016.0           2246.016667
296        shinomegami        2017.0              6.750000

[297 rows x 3 columns]


In [58]:
summary_playtime_per_year.columns

Index(['user_id', 'release_year', 'playtime_total_hours'], dtype='object')

## Carga de los dataframe
Se almacenan los dataframes en formato Parquet para optimizar la estructura de datos en el despliegue.

In [61]:
dfs = [max_playtime_by_genre_year, max_playtime_per_genre, summary_playtime_per_year]
# Nombres correspondientes a cada DataFrame
names = ['max_playtime_by_genre_year', 'max_playtime_per_genre', 'summary_playtime_per_year']

for df, name in zip(dfs, names):
    archivo = f'../Datasets/{name}.parquet'
    df.to_parquet(archivo)
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame 'max_playtime_by_genre_year' guardado como '../Datasets/max_playtime_by_genre_year.parquet'
DataFrame 'max_playtime_per_genre' guardado como '../Datasets/max_playtime_per_genre.parquet'
DataFrame 'summary_playtime_per_year' guardado como '../Datasets/summary_playtime_per_year.parquet'
